In [38]:
from search import *
import operator #Importamos operator para poder realizar operaciones con tuplas.

In [39]:
class MisioYCani(Problem):

    def __init__(self):
        initial_state = State.value_of(INITIAL_STATE)
        goal_state = State.value_of(GOAL_STATE)
        super().__init__(initial_state, goal_state)

    def actions(self, state):
        all_actions = self.get_all_actions()
        return self.get_valid_actions(state, all_actions)

    @staticmethod
    def get_all_actions():
        return {
            (1, 0, 1),
            (2, 0, 1),
            (0, 1, 1),
            (0, 2, 1),
            (1, 1, 1)
        }

    def get_valid_actions(self, state, all_actions):
        is_action_valid_lambda = self.get_is_action_valid_lambda(state)
        return set(filter(is_action_valid_lambda, all_actions))

    def get_is_action_valid_lambda(self, state):
        return lambda action: self.is_action_valid(state, action)

    def is_action_valid(self, state, action):
        operate = self.get_operation(state.boat)
        result = operate(state, action)

        return result.is_valid()

    def result(self, state, action):
        operate = self.get_operation(state.boat)
        return operate(state, action)

    @staticmethod
    def get_operation(boat):
        return operator.sub if boat == 1 else operator.add

In [40]:
class State:
    def __init__(self, missionaries, cannibals, boat):
        self.value = (missionaries, cannibals, boat)
        self.missionaries = missionaries
        self.cannibals = cannibals
        self.boat = boat

    @classmethod
    def value_of(cls, state):
        if not cls.__is_valid_tuple(state):
            raise ValueError(str(state) + " debe ser una tupla de 3 elementos.") #Esta funcion revisa si esta bien planteado el problema.
        return State(state[0], state[1], state[2])

    def is_valid(self):
        if contains_negative(self.value):
            return False
        elif self.__has_more_than_one_boat():
            return False
        elif self.__has_more_cannibals_than_missionaries():
            return False
        elif self.value > INITIAL_STATE:
            return False
        else:
            return True

    def __has_more_than_one_boat(self):
        return self.boat > 1

    def __has_more_cannibals_than_missionaries(self):
        return self.__more_cannibals_on_wrong_side() or self.__more_cannibals_on_right_side()

    def __more_cannibals_on_wrong_side(self):
        """Esta funcion regresa verdadero si hay mas canibales en el lado inicial"""
        return ((self.missionaries == 1 and self.cannibals == 3) or
                (self.missionaries == 2 and self.cannibals == 3) or
                (self.missionaries == 1 and self.cannibals == 2))

    def __more_cannibals_on_right_side(self):
        """Esta funcion regresa verdadero si hay mas canibales en el lado opuesto al inicial"""
        return ((self.missionaries == 2 and self.cannibals == 1) or
                (self.missionaries == 1 and self.cannibals == 0))
#Aqui se utiliza la libreria operator para utilizar las operaciones en las tuplas
    def __add__(self, other):
        result = self.__operate(other, operator.add)
        return self.value_of(result)

    def __sub__(self, other):
        result = self.__operate(other, operator.sub)
        return self.value_of(result)

    def __operate(self, other, operation):
        if self.__is_valid_tuple(other):
            return operate_on_tuples(self.value, other, operation)
        elif isinstance(other, State):
            return operate_on_tuples(self.value, other.value, operation)
        else:
            raise ValueError(self.__get_invalid_operand_error(other))

    @staticmethod
    def __is_valid_tuple(other):
        return isinstance(other, tuple) and len(other) == 3

    @staticmethod
    def __get_invalid_operand_error(other):
        return str(other) + " must be an instance of State or a tuple of length 3."

    def __repr__(self):
        return '<State {}>'.format(self.value)

    def __str__(self):
        return '<State {}>'.format(self.value)

    def __lt__(self, other):
        self.__ensure_instance_of_state(other)
        return self.value < other.value

    def __eq__(self, other):
        return isinstance(other, State) and self.value == other.value

    def __hash__(self):
        return hash(self.value)

    @staticmethod
    def __ensure_instance_of_state(other):
        if not isinstance(other, State):
            raise ValueError(str(other) + " must be an instance of State")


In [41]:
def add_tuples(a, b):
    return operate_on_tuples(a, b, operator.add)


def subtract_tuples(a, b):
    return operate_on_tuples(a, b, operator.sub)


def operate_on_tuples(a, b, operation):
    return tuple(map(operation, a, b))

def contains_negative(collection):
    return any(n < 0 for n in collection)

# Aqui se viene lo chido


In [42]:
INITIAL_STATE = (3, 3, 1)
GOAL_STATE = (0, 0, 0)

In [43]:
from search import iterative_deepening_search

def print_path(path):
    for node in path:
        print(node.state.value)

problem = MisioYCani()
result = iterative_deepening_search(problem)
result2 = depth_limited_search(problem, limit=15)
print_path(result.path())


(3, 3, 1)
(3, 1, 0)
(3, 2, 1)
(3, 0, 0)
(3, 1, 1)
(2, 0, 0)
(2, 2, 1)
(0, 2, 0)
(0, 3, 1)
(0, 1, 0)
(0, 2, 1)
(0, 0, 0)


In [44]:
print_path(result2.path()) #Dependiendo el limite del depth limited se realizan ciclos inescesarios en el algoritmo.

(3, 3, 1)
(3, 2, 0)
(3, 3, 1)
(3, 2, 0)
(3, 3, 1)
(3, 1, 0)
(3, 2, 1)
(3, 0, 0)
(3, 1, 1)
(2, 0, 0)
(2, 2, 1)
(0, 2, 0)
(0, 3, 1)
(0, 1, 0)
(0, 2, 1)
(0, 0, 0)
